In [4]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Sample dataset
data = {
    "Address": ["1600 Amphitheatre Parkway", "1 Infinite Loop", "Oostzijde"],
    "City": ["Mountain View", "Cupertino", "ZAANDAM"],
    "PostalCode": ["94043", "95014", "1502 BH"],
    "has_deft": [True, False, True]
}

df = pd.DataFrame(data)

# Normalize address format (remove extra spaces, make it title case)
df["Full_Address"] = df["Address"].str.title() + ", " + df["City"].str.title() + ", " + df["PostalCode"]

# Initialize geolocator
geolocator = Nominatim(user_agent="geo_mapper")

# Function to get coordinates
def get_coordinates(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

# Apply geocoding
df["Latitude"], df["Longitude"] = zip(*df["Full_Address"].apply(get_coordinates))

# Remove rows with missing coordinates
df = df.dropna(subset=["Latitude", "Longitude"])

# Create a folium map centered on the first location
map_center = [df["Latitude"].mean(), df["Longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=2)

# Add points to the map
for _, row in df.iterrows():
    color = "blue" if row["has_deft"] else "red"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=row["Full_Address"]
    ).add_to(m)



# Display map in Jupyter Notebook (if applicable)
m
